In [1]:
GLOBAL_SEED = 1000

In [2]:
from az_common_funcs import *
# from preprocessing_utils import preprocessing_utils

2024-04-01 21:37:06,190 :: [INFO] :: Logger initialized WITHOUT file handler


In [3]:
import os

In [4]:
df = pd.read_json(f"{data_dir}/S4_sentence_len_matched_labelled_pairs_F1.jsonl", lines=True)

In [5]:
df.head()

,review_id,sent_gte_3,cognitive_score,affective_score,spatial_embedding_score,temporal_embedding_score,narrativity_label,doc_a,review_length,review_num_sentences
0,7778460,True,0.333333,0.333333,2,2,good,Yleen made our week long visit a very enjoyabl...,47,3
1,26416346,True,0.333333,0.333333,2,2,good,Clean and convenient place to stay while we vi...,57,3
2,28564976,True,0.333333,0.333333,2,2,good,I had a wonderful time and was amazed at how m...,62,3
3,32947846,True,0.333333,0.333333,2,2,good,"When I come back to San Antonio, I would 100% ...",53,3
4,38709059,True,0.333333,0.333333,2,2,good,"Great location, only a 10 minute Lyft trip to ...",68,3


In [6]:
df.narrativity_label.value_counts()

narrativity_label
good    12876
bad     12876
Name: count, dtype: int64

In [6]:
pu = preprocessing_utils()

In [7]:
df['review_length'] = df['doc_a'].multicore_apply_by_chunks(pu.calculate_word_length, 8, 16)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3212/3212 [00:02<00:00, 1247.05it/s]


In [8]:
df.groupby('narrativity_label')['review_length'].describe()

,count,mean,std,min,25%,50%,75%,max
narrativity_label,,,,,,,,
bad,12876.0,50.700916,44.348114,3.0,19.0,33.0,71.0,461.0
good,12876.0,135.517707,97.241756,20.0,67.0,109.0,172.0,1217.0


In [66]:
df.shape

(25752, 11)

### Process NER

In [9]:
df['NER'] = df['doc_a'].progress_apply(pu.stanza_ner)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25752/25752 [17:49<00:00, 24.09it/s]


In [11]:
df.to_parquet(f"{temp_dir}/s4_feature_creation_ner_checkpoint.parquet", index=False)

### Process Gold Score Prediction for Sentence Coherence

In [62]:
import torch

In [63]:
gold_score_predictions = torch.load(os.path.join("/home/azikre/aadil/github/Transformer-Models-for-Text-Coherence-Assessment/dataset_processor/predictions", f"predictions_2024_03_19.pt"))

In [65]:
len(gold_score_predictions)

12876

### Process Taaco Cohesion

In [38]:
# cmd(f"cp -r ../github/TAACO ./")

In [7]:
from TAACOnoGUI import runTAACO

In [8]:
class TAACO_utils:
    def __init__(self):
        print("init TAACO utils")
        
    def save_df_rows(self,df, id_col, text_col):
        cmd(f"rm {temp_dir}/airbnb_reviews/*.txt")
        for row in df.itertuples():
            with open(f"{temp_dir}/airbnb_reviews/review_{getattr(row, id_col)}.txt", mode = 'w') as fp:
                fp.write(getattr(row, text_col))
                
    def run_TAACO(self, sampleVars=None, save_as="runTaaco_no_name.csv"):
        if not sampleVars: 
            raise NotImplementedError("Pass SampleVars. Default not defined.")
        
        runTAACO(f"{temp_dir}/airbnb_reviews/", f"{temp_dir}/{save_as}", sampleVars)
        print(f"TAACO finished Running. Results saved at: {temp_dir}/{save_as}")

In [9]:
tu = TAACO_utils()

init TAACO utils


In [61]:
tu.save_df_rows(df, 'review_id', 'doc_a')

Running Command :: rm /home/azikre/aadil/temp//airbnb_reviews/*.txt
output ::  
error ::  
return code ::  0


In [11]:
sampleVars = {'sourceKeyOverlap': False,
 'sourceLSA': False,
 'sourceLDA': False,
 'sourceWord2vec': False,
 'wordsAll': True,
 'wordsContent': True,
 'wordsFunction': True,
 'wordsNoun': True,
 'wordsPronoun': True,
 'wordsArgument': True,
 'wordsVerb': True,
 'wordsAdjective': True,
 'wordsAdverb': True,
 'overlapSentence': True, # Calculate sentence to sentence overlap
 'overlapParagraph': False,
 'overlapAdjacent': True, # Calculate overlap for adjacent sections (sentences or paragraphs)
 'overlapAdjacent2': True,
 'otherTTR': False,
 'otherConnectives': True, # Calculate connective indicidence indices
 'otherGivenness': True, # Calculate givenness indices
 'overlapLSA': True,
 'overlapLDA': True,
 'overlapWord2vec': True,
 'overlapSynonym': True,
 'overlapNgrams': True,
 'outputTagged': False,
 'outputDiagnostic': False}

# Run TAACO on a folder of texts ("ELLIPSE_Sample/"), give the output file a name ("packageTest.csv), provide output for particular indices/options (as defined in sampleVars)
tu.run_TAACO(sampleVars)

Loading Spacy
Loading Spacy Model
Starting TAACO...
Loading LSA vector space...
Loading LDA vector space...
Loading word2vec vector space...
outdir: /home/azikre/aadil/temp//runTaaco_no_name.csv
key_out_dir: /home/azikre/aadil/temp/
TAACO is processing 1 of 25756 files
TAACO is processing 2 of 25756 files
TAACO is processing 3 of 25756 files
TAACO is processing 4 of 25756 files
TAACO is processing 5 of 25756 files
TAACO is processing 6 of 25756 files
TAACO is processing 7 of 25756 files
TAACO is processing 8 of 25756 files
TAACO is processing 9 of 25756 files
TAACO is processing 10 of 25756 files
TAACO is processing 11 of 25756 files
TAACO is processing 12 of 25756 files
TAACO is processing 13 of 25756 files
TAACO is processing 14 of 25756 files
TAACO is processing 15 of 25756 files
TAACO is processing 16 of 25756 files
TAACO is processing 17 of 25756 files
TAACO is processing 18 of 25756 files
TAACO is processing 19 of 25756 files
TAACO is processing 20 of 25756 files
TAACO is process

TAACO is processing 209 of 25756 files
TAACO is processing 210 of 25756 files
TAACO is processing 211 of 25756 files
TAACO is processing 212 of 25756 files
TAACO is processing 213 of 25756 files
TAACO is processing 214 of 25756 files
TAACO is processing 215 of 25756 files
TAACO is processing 216 of 25756 files
TAACO is processing 217 of 25756 files
TAACO is processing 218 of 25756 files
TAACO is processing 219 of 25756 files
TAACO is processing 220 of 25756 files
TAACO is processing 221 of 25756 files
TAACO is processing 222 of 25756 files
TAACO is processing 223 of 25756 files
TAACO is processing 224 of 25756 files
TAACO is processing 225 of 25756 files
TAACO is processing 226 of 25756 files
TAACO is processing 227 of 25756 files
TAACO is processing 228 of 25756 files
TAACO is processing 229 of 25756 files
TAACO is processing 230 of 25756 files
TAACO is processing 231 of 25756 files
TAACO is processing 232 of 25756 files
TAACO is processing 233 of 25756 files
TAACO is processing 234 o

TAACO is processing 426 of 25756 files
TAACO is processing 427 of 25756 files
TAACO is processing 428 of 25756 files
TAACO is processing 429 of 25756 files
TAACO is processing 430 of 25756 files
TAACO is processing 431 of 25756 files
TAACO is processing 432 of 25756 files
TAACO is processing 433 of 25756 files
TAACO is processing 434 of 25756 files
TAACO is processing 435 of 25756 files
TAACO is processing 436 of 25756 files
TAACO is processing 437 of 25756 files
TAACO is processing 438 of 25756 files
TAACO is processing 439 of 25756 files
TAACO is processing 440 of 25756 files
TAACO is processing 441 of 25756 files
TAACO is processing 442 of 25756 files
TAACO is processing 443 of 25756 files
TAACO is processing 444 of 25756 files
TAACO is processing 445 of 25756 files
TAACO is processing 446 of 25756 files
TAACO is processing 447 of 25756 files
TAACO is processing 448 of 25756 files
TAACO is processing 449 of 25756 files
TAACO is processing 450 of 25756 files
TAACO is processing 451 o

TAACO is processing 644 of 25756 files
TAACO is processing 645 of 25756 files
TAACO is processing 646 of 25756 files
TAACO is processing 647 of 25756 files
TAACO is processing 648 of 25756 files
TAACO is processing 649 of 25756 files
TAACO is processing 650 of 25756 files
TAACO is processing 651 of 25756 files
TAACO is processing 652 of 25756 files
TAACO is processing 653 of 25756 files
TAACO is processing 654 of 25756 files
TAACO is processing 655 of 25756 files
TAACO is processing 656 of 25756 files
TAACO is processing 657 of 25756 files
TAACO is processing 658 of 25756 files
TAACO is processing 659 of 25756 files
TAACO is processing 660 of 25756 files
TAACO is processing 661 of 25756 files
TAACO is processing 662 of 25756 files
TAACO is processing 663 of 25756 files
TAACO is processing 664 of 25756 files
TAACO is processing 665 of 25756 files
TAACO is processing 666 of 25756 files
TAACO is processing 667 of 25756 files
TAACO is processing 668 of 25756 files
TAACO is processing 669 o

TAACO is processing 863 of 25756 files
TAACO is processing 864 of 25756 files
TAACO is processing 865 of 25756 files
TAACO is processing 866 of 25756 files
TAACO is processing 867 of 25756 files
TAACO is processing 868 of 25756 files
TAACO is processing 869 of 25756 files
TAACO is processing 870 of 25756 files
TAACO is processing 871 of 25756 files
TAACO is processing 872 of 25756 files
TAACO is processing 873 of 25756 files
TAACO is processing 874 of 25756 files
TAACO is processing 875 of 25756 files
TAACO is processing 876 of 25756 files
TAACO is processing 877 of 25756 files
TAACO is processing 878 of 25756 files
TAACO is processing 879 of 25756 files
TAACO is processing 880 of 25756 files
TAACO is processing 881 of 25756 files
TAACO is processing 882 of 25756 files
TAACO is processing 883 of 25756 files
TAACO is processing 884 of 25756 files
TAACO is processing 885 of 25756 files
TAACO is processing 886 of 25756 files
TAACO is processing 887 of 25756 files
TAACO is processing 888 o

...

TAACO is processing 24939 of 25756 files
TAACO is processing 24940 of 25756 files
TAACO is processing 24941 of 25756 files
TAACO is processing 24942 of 25756 files
TAACO is processing 24943 of 25756 files
TAACO is processing 24944 of 25756 files
TAACO is processing 24945 of 25756 files
TAACO is processing 24946 of 25756 files
TAACO is processing 24947 of 25756 files
TAACO is processing 24948 of 25756 files
TAACO is processing 24949 of 25756 files
TAACO is processing 24950 of 25756 files
TAACO is processing 24951 of 25756 files
TAACO is processing 24952 of 25756 files
TAACO is processing 24953 of 25756 files
TAACO is processing 24954 of 25756 files
TAACO is processing 24955 of 25756 files
TAACO is processing 24956 of 25756 files
TAACO is processing 24957 of 25756 files
TAACO is processing 24958 of 25756 files
TAACO is processing 24959 of 25756 files
TAACO is processing 24960 of 25756 files
TAACO is processing 24961 of 25756 files
TAACO is processing 24962 of 25756 files
TAACO is process

TAACO is processing 25141 of 25756 files
TAACO is processing 25142 of 25756 files
TAACO is processing 25143 of 25756 files
TAACO is processing 25144 of 25756 files
TAACO is processing 25145 of 25756 files
TAACO is processing 25146 of 25756 files
TAACO is processing 25147 of 25756 files
TAACO is processing 25148 of 25756 files
TAACO is processing 25149 of 25756 files
TAACO is processing 25150 of 25756 files
TAACO is processing 25151 of 25756 files
TAACO is processing 25152 of 25756 files
TAACO is processing 25153 of 25756 files
TAACO is processing 25154 of 25756 files
TAACO is processing 25155 of 25756 files
TAACO is processing 25156 of 25756 files
TAACO is processing 25157 of 25756 files
TAACO is processing 25158 of 25756 files
TAACO is processing 25159 of 25756 files
TAACO is processing 25160 of 25756 files
TAACO is processing 25161 of 25756 files
TAACO is processing 25162 of 25756 files
TAACO is processing 25163 of 25756 files
TAACO is processing 25164 of 25756 files
TAACO is process

TAACO is processing 25342 of 25756 files
TAACO is processing 25343 of 25756 files
TAACO is processing 25344 of 25756 files
TAACO is processing 25345 of 25756 files
TAACO is processing 25346 of 25756 files
TAACO is processing 25347 of 25756 files
TAACO is processing 25348 of 25756 files
TAACO is processing 25349 of 25756 files
TAACO is processing 25350 of 25756 files
TAACO is processing 25351 of 25756 files
TAACO is processing 25352 of 25756 files
TAACO is processing 25353 of 25756 files
TAACO is processing 25354 of 25756 files
TAACO is processing 25355 of 25756 files
TAACO is processing 25356 of 25756 files
TAACO is processing 25357 of 25756 files
TAACO is processing 25358 of 25756 files
TAACO is processing 25359 of 25756 files
TAACO is processing 25360 of 25756 files
TAACO is processing 25361 of 25756 files
TAACO is processing 25362 of 25756 files
TAACO is processing 25363 of 25756 files
TAACO is processing 25364 of 25756 files
TAACO is processing 25365 of 25756 files
TAACO is process

TAACO is processing 25542 of 25756 files
TAACO is processing 25543 of 25756 files
TAACO is processing 25544 of 25756 files
TAACO is processing 25545 of 25756 files
TAACO is processing 25546 of 25756 files
TAACO is processing 25547 of 25756 files
TAACO is processing 25548 of 25756 files
TAACO is processing 25549 of 25756 files
TAACO is processing 25550 of 25756 files
TAACO is processing 25551 of 25756 files
TAACO is processing 25552 of 25756 files
TAACO is processing 25553 of 25756 files
TAACO is processing 25554 of 25756 files
TAACO is processing 25555 of 25756 files
TAACO is processing 25556 of 25756 files
TAACO is processing 25557 of 25756 files
TAACO is processing 25558 of 25756 files
TAACO is processing 25559 of 25756 files
TAACO is processing 25560 of 25756 files
TAACO is processing 25561 of 25756 files
TAACO is processing 25562 of 25756 files
TAACO is processing 25563 of 25756 files
TAACO is processing 25564 of 25756 files
TAACO is processing 25565 of 25756 files
TAACO is process

TAACO is processing 25747 of 25756 files
TAACO is processing 25748 of 25756 files
TAACO is processing 25749 of 25756 files
TAACO is processing 25750 of 25756 files
TAACO is processing 25751 of 25756 files
TAACO is processing 25752 of 25756 files
TAACO is processing 25753 of 25756 files
TAACO is processing 25754 of 25756 files
TAACO is processing 25755 of 25756 files
TAACO is processing 25756 of 25756 files
Processed 25756 Files
TAACO finished Running. Results saved at: /home/azikre/aadil/temp//runTaaco_no_name.csv


In [11]:
df_taaco_res = pd.read_csv(f"/home/azikre/aadil/temp/runTaaco_no_name.csv")

In [14]:
df_taaco_res['review_id'] = df_taaco_res.Filename.apply(lambda x : x.split(".")[0].split("_")[1])